In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import torch
import os
from transformers import MarianMTModel, MarianTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Tạo thư mục cache và lưu tokenizer
os.makedirs("./cache", exist_ok=True)
os.makedirs("./tokenizer", exist_ok=True)

In [ ]:
# Tải tokenizer và mô hình
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# Chuyển sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(53685, 512, padding_idx=53684)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(53685, 512, padding_idx=53684)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
# Đọc file
def load_parallel_data(en_file, vi_file):
    with open(en_file, 'r', encoding='utf-8') as f_en:
        en_sentences = [line.strip() for line in f_en]
    with open(vi_file, 'r', encoding='utf-8') as f_vi:
        vi_sentences = [line.strip() for line in f_vi]
    assert len(en_sentences) == len(vi_sentences), "Số câu không khớp!"
    return {"en": en_sentences, "vi": vi_sentences}

en_file = "/kaggle/input/englishvietnamese-translation/en_sents"  # Thay bằng đường dẫn
vi_file = "/kaggle/input/englishvietnamese-translation/vi_sents"  # Thay bằng đường dẫn
data = load_parallel_data(en_file, vi_file)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/englishvietnamese-translation/en.txt'

In [ ]:
dataset = Dataset.from_dict(data)

In [ ]:
print(f"Số câu: {len(data['en'])}")

Số câu: 254090


In [ ]:
# Tiền xử lý
def preprocess_function(examples):
    inputs = examples["en"]
    targets = examples["vi"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize với cache
tokenized_dataset = dataset.map(preprocess_function, batched=True, cache_file_name="./cache/tokenized_cache.arrow")

Map:   0%|          | 0/254090 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Chia dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [ ]:
# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=1000,
    save_total_limit=2,
    fp16=True,  # Mixed precision (GPU only)
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Khởi tạo Trainer với EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping
)

In [ ]:
# Huấn luyện mô hình
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.040500,0.034044
2000,0.034200,0.030785
3000,0.031700,0.029101
4000,0.029400,0.028157
5000,0.028600,0.027503
6000,0.027400,0.027127
7000,0.026800,0.026903


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead

TrainOutput(global_step=7940, training_loss=0.038590788180641926, metrics={'train_runtime': 15508.0993, 'train_samples_per_second': 65.537, 'train_steps_per_second': 0.512, 'total_flos': 3.443531502845952e+16, 'train_loss': 0.038590788180641926, 'epoch': 4.997167138810198})

In [ ]:
# Lưu mô hình dưới dạng .pt
torch.save(model.state_dict(), "/kaggle/working/translate.pt")

# Lưu tokenizer
tokenizer.save_pretrained("/kaggle/working/tokenizer")

('/kaggle/working/tokenizer/tokenizer_config.json',
 '/kaggle/working/tokenizer/special_tokens_map.json',
 '/kaggle/working/tokenizer/vocab.json',
 '/kaggle/working/tokenizer/source.spm',
 '/kaggle/working/tokenizer/target.spm',
 '/kaggle/working/tokenizer/added_tokens.json')

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
 # Install the missing sacrebleu library
import torch
import os
from transformers import MarianMTModel, MarianTokenizer
import sacrebleu # Now the import should work

def test_translation(sentence, model_path="/content/drive/MyDrive/DeepLearning /Translate/translate.pt", tokenizer_path="/content/drive/MyDrive/DeepLearning /Translate/tokenizer"):
    # Kiểm tra GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Kiểm tra file
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"File {model_path} không tồn tại!")
    if not os.path.exists(tokenizer_path):
        raise FileNotFoundError(f"Thư mục {tokenizer_path} không tồn tại!")

    # Tải tokenizer
    tokenizer = MarianTokenizer.from_pretrained(tokenizer_path)

    # Tải mô hình
    model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-vi")
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Dịch câu
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    # Tham chiếu tiếng Việt (danh sách tham chiếu)
    references_sacrebleu = [
        "Tôi có thể đấm anh được không?"
    ]

    # Tính BLEU với sacrebleu
    hypothesis_sacrebleu = [translated_text]
    bleu_sacrebleu_obj = sacrebleu.corpus_bleu(hypothesis_sacrebleu, [references_sacrebleu])
    bleu_sacrebleu = bleu_sacrebleu_obj.score

    # In kết quả
    print(f"Câu gốc (Anh): {sentence}")
    print(f"Câu dịch (Việt): {translated_text}")
    # print(f"BLEU Score (sacrebleu): {bleu_sacrebleu:.4f}")

    return translated_text, bleu_sacrebleu


In [ ]:

# Test câu
sentence = "The weather is very nice today. I want to go to the park and read a book under a tree. Maybe I will also bring some snacks and enjoy the sunshine."
translated_text, bleu_sacrebleu = test_translation(sentence)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Câu gốc (Anh): The weather is very nice today. I want to go to the park and read a book under a tree. Maybe I will also bring some snacks and enjoy the sunshine.
Câu dịch (Việt): hôm nay trời rất đẹp tôi muốn đi đến công viên và đọc một cuốn sách dưới gốc cây có lẽ tôi cũng sẽ mang theo một ít đồ ăn nhẹ và thưởng thức ánh nắng mặt trời


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
# !pip install gradio sacrebleu

import torch
import os
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer
import sacrebleu

# Khởi tạo mô hình và tokenizer (tải trước để không load lại nhiều lần)
model_path = "/content/drive/MyDrive/DeepLearning /Translate/translate.pt"
tokenizer_path = "/content/drive/MyDrive/DeepLearning /Translate/tokenizer"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"File {model_path} không tồn tại!")
if not os.path.exists(tokenizer_path):
    raise FileNotFoundError(f"Thư mục {tokenizer_path} không tồn tại!")

tokenizer = MarianTokenizer.from_pretrained(tokenizer_path)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-vi")
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Hàm dịch dùng cho Gradio
def translate_input(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    # Tính BLEU demo nếu muốn
    # references = ["Tôi có thể đấm anh được không?"]
    # bleu = sacrebleu.corpus_bleu([translated_text], [references]).score

    return f"{translated_text}"

# Giao diện Gradio
interface = gr.Interface(
    fn=translate_input,
    inputs=gr.Textbox(lines=3, label="Nhập câu tiếng Anh"),
    outputs=gr.Textbox(label="Kết quả dịch"),
    title="Dịch tiếng Anh sang tiếng Việt",
    description="Nhập câu tiếng Anh và nhấn 'Submit' để dịch."
)

interface.launch()


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd69d20d1bb8312dbb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import os
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer

# Load mô hình
model_path = "/content/drive/MyDrive/DeepLearning /Translate/translate.pt"
tokenizer_path = "/content/drive/MyDrive/DeepLearning /Translate/tokenizer"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"File {model_path} không tồn tại!")
if not os.path.exists(tokenizer_path):
    raise FileNotFoundError(f"Thư mục {tokenizer_path} không tồn tại!")

tokenizer = MarianTokenizer.from_pretrained(tokenizer_path)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-vi")
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Danh sách các cụm chuyên ngành cần bảo vệ
protected_phrases = {
    "Convolutional Neural Networks (CNNs)": "#CNN_DESC#",
    "Recurrent Neural Networks (RNNs)": "#RNN_DESC#",
    "Long Short-Term Memory (LSTM)": "#LSTM_DESC#",
    "CNNs": "#CNN#",
    "RNNs": "#RNN#",
    "LSTM": "#LSTM#",
    "GPUs": "#GPU#",
    "AI": "#AI#",
    "NLP": "#NLP#"
}

# Hàm thay thế cụm từ trước khi dịch
def protect_terms(text):
    for phrase, tag in protected_phrases.items():
        text = text.replace(phrase, tag)
    return text

# Hàm khôi phục cụm từ sau khi dịch
def restore_terms(text):
    for phrase, tag in protected_phrases.items():
        text = text.replace(tag, phrase)
    return text

# Hàm dịch chính
def translate_input(sentence):
    protected_sentence = protect_terms(sentence)
    inputs = tokenizer(protected_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    final_text = restore_terms(translated_text)
    return final_text

# Giao diện Gradio
interface = gr.Interface(
    fn=translate_input,
    inputs=gr.Textbox(lines=6, label="Nhập đoạn tiếng Anh"),
    outputs=gr.Textbox(label="Bản dịch tiếng Việt"),
    title="Dịch tiếng Anh sang tiếng Việt (giữ nguyên thuật ngữ chuyên ngành)",
    description="Giữ nguyên các cụm từ như 'Convolutional Neural Networks (CNNs)', 'RNNs', 'LSTM' khi dịch."
)

interface.launch()


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a72824360b943b8c79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import os
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer

# Khởi tạo mô hình và tokenizer
model_path = "/content/drive/MyDrive/DeepLearning /Translate/translate.pt"
tokenizer_path = "/content/drive/MyDrive/DeepLearning /Translate/tokenizer"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"File {model_path} không tồn tại!")
if not os.path.exists(tokenizer_path):
    raise FileNotFoundError(f"Thư mục {tokenizer_path} không tồn tại!")

tokenizer = MarianTokenizer.from_pretrained(tokenizer_path)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-vi")
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Hàm dịch
def translate_input(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Giao diện Gradio đẹp hơn
with gr.Blocks(theme=gr.themes.Soft()) as interface:
    gr.Markdown(
        """
        # 🌐 Translator: English ➜ Vietnamese
        Nhập một câu tiếng Anh vào bên dưới và nhấn **Dịch** để nhận bản dịch tiếng Việt.
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="✏️ Câu tiếng Anh", lines=3, placeholder="Nhập nội dung cần dịch...")
            translate_button = gr.Button("🔁 Dịch", variant="primary")
        with gr.Column():
            output_text = gr.Textbox(label="📘 Bản dịch tiếng Việt", lines=3, interactive=False)

    translate_button.click(fn=translate_input, inputs=input_text, outputs=output_text)

interface.launch()


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0b4492fe342865c1eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
